In [172]:
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from string import printable
import unicodedata



base_url = "https://spb.hh.ru/resume/"

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}

In [43]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [203]:
def salary(content):
    soup = BeautifulSoup(content)
    text = soup.find('span', 
        {
            'class':'resume-block__salary resume-block__title-text_salary',                    "data-qa":"resume-block-salary"
        }).text
    text = unicodedata.normalize('NFKD', text)
    salary_list = [char for char in text if char.isdigit()]
    return int(''.join(salary_list))

In [219]:
def work_exp(content):
    soup = BeautifulSoup(content)
    text = soup.find('span', {
            'class':'resume-block__title-text resume-block__title-text_sub'
        }).text
    
    text = unicodedata.normalize('NFKD', text)
    return text.replace('Опыт работы ', '')

In [236]:
def drive_exp(content):
    soup = BeautifulSoup(content)
    div = soup.find_all('div', {
            'data-qa':'resume-block-driver-experience'
        })[0]
    text = div.find_all('div', {'class': 'resume-block-item-gap'})[0].text
    text = unicodedata.normalize('NFKD', text)
    return text.replace('Опыт работы ', '')

In [240]:
def education(content):
    soup = BeautifulSoup(content)
    div = soup.find_all('div', {
            'data-qa':'resume-block-education'
        })[0]
    text = div.find_all('div', {'class': 'resume-block-item-gap'})[0].text
    text = unicodedata.normalize('NFKD', text)
    return text.replace('Опыт работы ', '')

In [242]:
term_parser = {
    'зарплата': salary,
    'зп': salary,
    'опыт работы': work_exp,
    'опыт вождения': drive_exp,
    'образование': education
}

In [243]:
def parser(question_terms, uid):
    #content = download.resume(uid)
    driver.get(base_url + uid)
    content = driver.page_source
    output = {}
    for name in question_terms:
        output[name] = term_parser[name](content)
    return output

In [244]:
parser(['зарплата', 'опыт работы', 'опыт вождения', 'образование'], 'fc6280cb00051725840039ed1f574c52506830')

{'зарплата': 40000,
 'опыт работы': '9 месяцев',
 'опыт вождения': 'Права категории B',
 'образование': 'Среднее образование'}